# Entrenamiento para clasificacion automatica de textos clinicos
Intentaremos entrenar un modelo para clasificar una descripcion ingresada, con su respectivo nombre de enfermedad

In [1]:
## Importamos las librerias de tensorflow keras para el entrenamiento del modelo, y pandas con numpy para el manejo del dataset

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

In [2]:
## Leemos el csv del dataset y lo almacenamos en una variable, la cual mostraremos a continuacion

dataset_entrenamiento = pd.read_csv("dataset.csv")
dataset_entrenamiento

,short_description,icd_code,long_title
0,Hospital observation services,0049,"Shigellosis, unspecified"
1,Hospital observation services,0059,"Food poisoning, unspecified"
2,Hospital observation services,0071,Giardiasis
3,Hospital observation services,0074,Cryptosporidiosis
4,Hospital observation services,00800,"Intestinal infection due to E. coli, unspecified"
...,...,...,...
37807,Musculoskeletal system,Z9981,Dependence on supplemental oxygen
37808,Nervous system,Z9981,Dependence on supplemental oxygen
37809,Respiratory system,Z9981,Dependence on supplemental oxygen
37810,Urinary system,Z9981,Dependence on supplemental oxygen


In [3]:
## Asignamos las entradas y salidas para nuestro modelo de entrenamiento. En este caso usaremos la columna de short_description
## para las entradas, y como salida usaremos su respectivo icd_code

entradas = np.array(dataset_entrenamiento["short_description"])
salidas = np.array(dataset_entrenamiento["icd_code"])

In [4]:
## Mostramos los registros almacenados en la variable 'entradas'

entradas

array(['Hospital observation services', 'Hospital observation services',
       'Hospital observation services', ..., 'Respiratory system',
       'Urinary system', 'Hospital observation services'], dtype=object)

In [5]:
#Mostramos los registros almacenados en la variable 'salidas'

salidas

array(['0049   ', '0059   ', '0071   ', ..., 'Z9981  ', 'Z9981  ',
       'Z9989  '], dtype=object)

In [6]:
## Ahora, para poder entrenar el modelo, debemos transformar las cadenas de caracteres en valores numericos. Usaremos la funcion
## one_hot() de keras para asignarle a cada palabra un valor numerico aleatorio entre 0 y 5000

vocab_size = 5000
cod_entradas = [one_hot(d, vocab_size) for d in entradas]
cod_entradas

[[4919, 4302, 3918],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [673, 1901],
 [4919, 4302, 3918],
 [542, 1901],
 [4919, 4302, 3918],
 [1684],
 [1684, 1901],
 [673, 1901],
 [2736, 3973, 1901],
 [4919, 4302, 3918],
 [220, 1901],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [673, 1901],
 [4919, 4302, 3918],
 [542, 1901],
 [220, 1901],
 [53, 1901],
 [673, 1901],
 [4919, 4302, 3918],
 [673, 1901],
 [4919, 4302, 3918],
 [542, 1901],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [1684, 1901],
 [4919, 4302, 3918],
 [1684],
 [673, 1901],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [4780, 1901],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [4780, 1901],
 [542, 1901],
 [220, 1901],
 [673, 1901],
 [4919, 4302, 3918],
 [53, 1901],
 [4919, 4302, 3918],
 [4919, 4302, 3918],
 [1626, 1901],
 [4919, 4302, 3918],
 [673, 1901],
 [1684, 1901],
 [673, 1901],
 [4919, 4302, 3918],
 [53, 1901],
 [4919, 4302, 3918],

In [7]:
##Buscamos el valor  en "short_description" que tenga el mayor numero de palabras para definir un largo maximo

lenghts = dataset_entrenamiento['short_description'].str.len()
argmax = np.where(lenghts == lenghts.max())[0]
dataset_entrenamiento.iloc[argmax]

,short_description,icd_code,long_title
98,"Hydration,therapeutic,prophylactic,diagnostic ...",04149,Other and unspecified Escherichia coli [E. coli]
225,"Hydration,therapeutic,prophylactic,diagnostic ...",07054,Chronic hepatitis C without mention of hepatic...
240,"Hydration,therapeutic,prophylactic,diagnostic ...",07070,Unspecified viral hepatitis C without hepatic ...
506,"Hydration,therapeutic,prophylactic,diagnostic ...",1550,"Malignant neoplasm of liver, primary"
760,"Hydration,therapeutic,prophylactic,diagnostic ...",1830,Malignant neoplasm of ovary
...,...,...,...
33379,"Hydration,therapeutic,prophylactic,diagnostic ...",V4573,Acquired absence of kidney
33736,"Hydration,therapeutic,prophylactic,diagnostic ...",V5811,Encounter for antineoplastic chemotherapy
33766,"Hydration,therapeutic,prophylactic,diagnostic ...",V5861,Long-term (current) use of anticoagulants
33841,"Hydration,therapeutic,prophylactic,diagnostic ...",V5866,Long-term (current) use of aspirin


In [8]:
##Buscamos el valor que se indica en la celda anterior para contar la cantidad de palabras que contiene

entradas[98]

'Hydration,therapeutic,prophylactic,diagnostic injections and infusions,and chemotherapy and other highly complex drug or highly complex biologic agent administration'

**Esta cadena que es la mas grande de esta columna, contiene 24 caracteres, asi que ese sera el maximo asignado para todas las demas.**

In [9]:
## Aqui asignaremos a cada una de las entradas que debe contener 24 valores (que son las palabras). Aquellas entradas que 
## contengan menos de 24 palabras, se rellenaran las faltantes con ceros

length = 24
padded_entr = pad_sequences(cod_entradas, maxlen = length, padding = 'pre')
print(padded_entr)

[[   0    0    0 ... 4919 4302 3918]
 [   0    0    0 ... 4919 4302 3918]
 [   0    0    0 ... 4919 4302 3918]
 ...
 [   0    0    0 ...    0   53 1901]
 [   0    0    0 ...    0 1626 1901]
 [   0    0    0 ... 4919 4302 3918]]


In [10]:
## Ahora transformaremos tambien los valores de las salidas, ya que tambien son caracteres y para manejarlos debemos utilizar 
## valroes numericos. Le asignamos un total de 24000, que es un valor aproximado de codigos que hay en el dataset, para que a
## cada uno se le asigne de manera aleatoria un valor entre 0 y 24000 con la funcion one_hot()

vocab_size_sal = 24000
cod_salidas = [one_hot(d, vocab_size_sal) for d in salidas]
cod_salidas

[[7368],
 [6338],
 [69],
 [23157],
 [8784],
 [15991],
 [15991],
 [15991],
 [23944],
 [11280],
 [11280],
 [11280],
 [11280],
 [11280],
 [11280],
 [3848],
 [7185],
 [11873],
 [18427],
 [18427],
 [18427],
 [18427],
 [18427],
 [6602],
 [6602],
 [5305],
 [5305],
 [5305],
 [10887],
 [19773],
 [11008],
 [23208],
 [23208],
 [22738],
 [22738],
 [22738],
 [11015],
 [22700],
 [22700],
 [21152],
 [6788],
 [6788],
 [6788],
 [6788],
 [1822],
 [1822],
 [1822],
 [22317],
 [7471],
 [7471],
 [2641],
 [10658],
 [7201],
 [7201],
 [7201],
 [7201],
 [2423],
 [12703],
 [8259],
 [17332],
 [17577],
 [9655],
 [9655],
 [9655],
 [19680],
 [19680],
 [19680],
 [19680],
 [19680],
 [8510],
 [10397],
 [10397],
 [14835],
 [14835],
 [14835],
 [14835],
 [14835],
 [14835],
 [14835],
 [14835],
 [14835],
 [6228],
 [6228],
 [6228],
 [6228],
 [11486],
 [11486],
 [11486],
 [11486],
 [20374],
 [20374],
 [20374],
 [20374],
 [15241],
 [15241],
 [15241],
 [15241],
 [15241],
 [15241],
 [15241],
 [15241],
 [15241],
 [15241],
 [15241

In [11]:
##Utilizamos la funcion concatenate de Numpy, ya que los valores en la salida estan ingresados como objetos, y solo nos permite
## ingresar valores numericos en la funcion para evaluar el modelo

cod_salidas = np.concatenate(cod_salidas).astype(None)
cod_salidas

array([ 7368.,  6338.,    69., ...,  8692.,  8692., 10201.])

In [12]:
## Definimos el modelo

modelo = Sequential()
modelo.add(Embedding(vocab_size, 8, input_length = length))
modelo.add(Flatten())
modelo.add(Dense(1, activation = 'sigmoid'))

In [13]:
## Compilamos el modelo

modelo.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
## Fitting model

modelo.fit(padded_entr, cod_salidas, epochs = 5)

#evaluar el modelo

modelo_perdidas, modelo_precision = modelo.evaluate(padded_entr, cod_salidas, verbose = 0)
print('Precision: %f' %(modelo_precision*100))

Epoch 1/5
1182/1182 [==============================] - 1s 650us/step - loss: -2638581.7500 - accuracy: 0.0000e+00
Epoch 2/5
1182/1182 [==============================] - 1s 627us/step - loss: -16966076.0000 - accuracy: 0.0000e+00
Epoch 3/5
1182/1182 [==============================] - 1s 637us/step - loss: -41927252.0000 - accuracy: 0.0000e+00
Epoch 4/5
1182/1182 [==============================] - 1s 610us/step - loss: -75418304.0000 - accuracy: 0.0000e+00
Epoch 5/5
1182/1182 [==============================] - 1s 624us/step - loss: -116458088.0000 - accuracy: 0.0000e+00
Precision: 0.000000


**Luego de entrenar el modelo, los resultados no son los esperados. No aumenta su precision y los valores de las perdidas son demasiado altos, por lo que el modelo no es capaz de predecir ni definir a que codigo icd podria corresponder una descripcion ingresada.**

**Esto es posible que se deba a que algunas de las descripciones ingresadas en la variable entradas se repiten en varios de los codigos icd, lo que quizas no le permitio al modelo poder encontrar alguna tendencia en estas descripciones.**